In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import pickle


# 搭建 简单 模型 
def build_simple_model(out_dims, img_size):
    inputs_dim = Input((img_size, img_size, 3))

    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',activation='relu')(inputs_dim)
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',activation='relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x_flat = Flatten()(x)
    fc1 = Dense(512,activation='relu')(x_flat)
    dp_1 = Dropout(0.4)(fc1)

    fc2 = Dense(out_dims)(dp_1)
    fc2 = Activation('softmax')(fc2)

    model = Model(inputs=inputs_dim, outputs=fc2)
    return model

def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f,encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32,32).transpose(0,2,3,1).astype("float") #channel last
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

ROOT = '/kaggle/input/dataset-cifar/cifar-10-batches-py'
X_train,y_train,X_test,y_test = load_CIFAR10(ROOT)

X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train, num_classes = 10)
y_test = np_utils.to_categorical(y_test, num_classes = 10)

X_train = X_train.astype(np.float32) #'float32'占4个字节
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.uint8) # 'uint8'占1个字节
y_test = y_test.astype(np.uint8)

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0)


simple_model = build_simple_model(10,32)
simple_model.summary()

ModuleNotFoundError: No module named 'keras'

In [ ]:
checkpointer = ModelCheckpoint(filepath='cifa_10_simple.hdf5', 
                              verbose=1, save_best_only=True) #保存最好模型权重
simple_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 5

history = simple_model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=epochs,callbacks=[checkpointer],verbose=1)

In [3]:
import matplotlib.pyplot as plt                        
# %matplotlib inline    

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined